In [ ]:
import cv2
import numpy as np

# Define camera properties (you need to adjust these values based on your camera and setup)
FOCAL_LENGTH = 1000  # Focal length of your camera's lens (in pixels)
KNOWN_OBJECT_WIDTH = 0.15  # Width of the object you know the size of (in meters)
MAX_DISTANCE = 5.0  # Maximum distance for detection (in meters)
MIN_DISTANCE_WARNING = 1.0  # Minimum distance for warning (in meters)

# Load YOLOv3-608 weights and configuration
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

# Load COCO class names
classes = []
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Get the output layer names
output_layer_names = net.getUnconnectedOutLayersNames()

# Initialize laptop camera
cap = cv2.VideoCapture(0)

# Define distance calculation function
def calculate_distance(w):
    # Calculate distance based on object width and focal length
    object_width = w
    distance = (KNOWN_OBJECT_WIDTH * FOCAL_LENGTH) / object_width
    return distance

# Define obstacle detection function
def detect_obstacle(x, y, w, h):
    # Calculate obstacle position and distance
    obstacle_x = x + w / 2
    obstacle_y = y + h / 2
    obstacle_distance = calculate_distance(w)

    # Display obstacle distance onto the bounding box
    cv2.putText(frame, f'Distance: {obstacle_distance:.2f} meters', (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Generate a warning if obstacle is within 1 meter
    if obstacle_distance <= MIN_DISTANCE_WARNING:
        cv2.putText(frame, 'WARNING: Too close!', (x, y - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Main loop for object detection
while True:
    # Capture frame from the camera
    ret, frame = cap.read()

    # Perform object detection
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    layeroutput = net.forward(output_layer_names)

    # Process the detected objects
    class_ids = []
    confidences = []
    boxes = []
    for out in layeroutput:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                # Store the object information
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to eliminate overlapping boxes
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels on the frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    colors = np.random.uniform(0, 255, size=(len(class_ids), 3))
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f'{label}: {confidence:.2f}', (x, y - 5), font, 0.5, color, 2)

            # Detect obstacles and add distance onto the bounding box
            detect_obstacle(x, y, w, h)

    # Display the frame
    cv2.imshow('Object Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the windows
cap.release()
cv2.destroyAllWindows()
